In [ ]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from gensim.models import FastText



In [ ]:
import itertools
from gensim.models.word2vec import Text8Corpus
#from glove import Corpus, Glove
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
%matplotlib inline

In [ ]:
# provide sql-like data manipulation tools. very handy.
import pandas as pd 
pd.options.mode.chained_assignment = None

# high dimensional vector computing library.
import numpy as np 
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim

# the word2vec model gensim class
from gensim.models.word2vec import Word2Vec 

# we'll talk about this down below
LabeledSentence = gensim.models.doc2vec.LabeledSentence 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

# a tweet tokenizer from nltk.
from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_df= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/AMI/dataset/taska.txt',header=None,sep='\t',names=['id','text','class'])
#valid_df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_dev.csv',header=None,names=['id','text','class'])

#test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_fb_gold.csv',header=None,names=['id','text','class'])


In [ ]:
train_df

,id,text,class
0,86426,@USER She should ask a few native Americans wh...,OFF
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,16820,Amazon is investigating Chinese employees who ...,NOT
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
13237,82921,@USER And why report this garbage. We don't g...,OFF
13238,27429,@USER Pussy,OFF


In [ ]:
# train.info()
# train.head()
# train['class'].value_counts()

In [ ]:
import re
def clean_text(comment):
    comment = comment.strip().strip('"')
    comment = comment.replace('_', ' ')
    comment = comment.replace("\\\\", "\\")
    comment = comment.replace('\\n', ' ')
    comment = comment.replace('\\n', ' ')
    comment = comment.lower()
    comment = re.sub(r'@[A-Za-z0-9]+','',comment)
    
    comment = re.sub('https?://[A-Za-z0-9./]+','',comment)
    commment = re.sub("[^a-zA-Z]", " ", comment)
    comment = re.sub(r'[^\w\s]','',comment)

    
    return comment

train_df['text'] = train_df['text'].fillna("something").values
train_df['ctext']=train_df['text'].apply(clean_text)
#train_df_df['ctext']=train_df_df['text']
#valid_df['ctext']=valid_df['text'].apply(clean_text)
#test_df['ctext']=test_df['text'].apply(clean_text)
train_df.head()


,id,text,class,ctext
0,86426,@USER She should ask a few native Americans wh...,OFF,she should ask a few native americans what th...
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,go home youre drunk maga trump2020 url
2,16820,Amazon is investigating Chinese employees who ...,NOT,amazon is investigating chinese employees who ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,someone shouldvetaken this piece of shit to a...
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,obama wanted liberals amp illegals to move i...


In [ ]:

#tr=train_df[train_df['text']=='something']
#tr.head()

In [ ]:
#train_df['text'] = train_df['text'].fillna("something").values
train_df['class'] = train_df['class'].map({'OFF': 0, 'NOT': 1})


In [ ]:
#train_df, test_df = train_test_split(train,test_size = 0.1)


In [ ]:
import nltk
nltk.download('punkt')
train_df['tokenized_sents'] = train_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)
#test_df['tokenized_sents'] = test_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train_df.head()

,id,text,class,ctext,tokenized_sents
0,86426,@USER She should ask a few native Americans wh...,0,she should ask a few native americans what th...,"[she, should, ask, a, few, native, americans, ..."
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,0,go home youre drunk maga trump2020 url,"[go, home, youre, drunk, maga, trump2020, url]"
2,16820,Amazon is investigating Chinese employees who ...,1,amazon is investigating chinese employees who ...,"[amazon, is, investigating, chinese, employees..."
3,62688,"@USER Someone should'veTaken"" this piece of sh...",0,someone shouldvetaken this piece of shit to a...,"[someone, shouldvetaken, this, piece, of, shit..."
4,43605,@USER @USER Obama wanted liberals &amp; illega...,1,obama wanted liberals amp illegals to move i...,"[obama, wanted, liberals, amp, illegals, to, m..."


In [ ]:
x_train=train_df['tokenized_sents']
#x_test=test_df['tokenized_sents']

y_train=train_df['class']
#y_test=test_df['class']


In [ ]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')

#x_test = labelizeTweets(x_test, 'TEST')


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
13240it [00:00, 234647.37it/s]


In [ ]:
#tweet_w2v = FastText(size=400, min_count=1,sg=1)
tweet_w2v = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/AMI/Pretrained Model/GoogleNews-vectors-negative300.bin', binary=True)
# tweet_w2v = Word2Vec(size=450, min_count=1,sg=1) #SKIPGRAM
#tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
#tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=len(x_train),epochs=10)


In [ ]:
#tweet_w2v.save('off-sg.vec')
#tweet_w2v.save('fb-en-w2vec-cbow.bin')
#tweet_w2v.wv.save_word2vec_format('fb-hi-w2vec-cbow.txt')
#tweet_w2v = gensim.models.KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/AMI/Pretrained_Google/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(min_df=1)
#matrix = vectorizer.fit_transform(x_train)
matrix = vectorizer.fit_transform(train_df['ctext'])

tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 20696


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#tweet_w2v=FastText.load('fb-en-fasttext-sg.vec')

In [ ]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += tweet_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
#tweet_w2v=Word2Vec.load('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow.bin')
#from gensim.models.wrappers import FastText

#tweet_w2v= FastText.load_fasttext_format('D:/ps/model/English/fasttext/coling-fasttext-eng-cbow')

n_dim=300
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)



#test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in tqdm(map(lambda x: x.words, x_test))])
#test_vecs_w2v = scale(test_vecs_w2v)




0it [00:00, ?it/s]
571it [00:00, 5703.63it/s]
1142it [00:00, 5677.57it/s]
1736it [00:00, 5794.22it/s]
2379it [00:00, 6039.24it/s]
2983it [00:00, 5712.75it/s]
3605it [00:00, 5874.08it/s]
4195it [00:00, 5584.37it/s]
4758it [00:00, 5414.40it/s]
5303it [00:00, 5368.86it/s]
5842it [00:01, 5158.53it/s]
6387it [00:01, 5241.75it/s]
6996it [00:01, 5487.41it/s]
7548it [00:01, 5482.79it/s]
8099it [00:01, 5275.62it/s]
8633it [00:01, 5292.59it/s]
9165it [00:01, 5211.36it/s]
9704it [00:01, 5261.03it/s]
10232it [00:01, 5253.48it/s]
10759it [00:01, 5255.63it/s]
11286it [00:02, 5169.36it/s]
11920it [00:02, 5512.48it/s]
12533it [00:02, 5692.30it/s]
13240it [00:02, 5504.02it/s]


###Shape
train_vecs_w2v.shape
test_vecs_w2v.shape
train_vecs_w2v[0]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB



clf = LogisticRegression()
#clf=LinearSVC()

clf.fit(train_vecs_w2v, y_train)

#print(clf.score(valid_vecs_w2v, y_valid))
#clf.score(test_vecs_w2v,y_test)


LogisticRegression()

In [ ]:
import pickle
filename = 'Logistic_regression.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
from sklearn import metrics

#y_pred_class = clf.predict(test_vecs_w2v)

print(metrics.accuracy_score(y_test, y_pred_class))

print(metrics.confusion_matrix(y_test, y_pred_class))
print (metrics.precision_score(y_test, y_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, y_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(y_test, y_pred_class,target_names=['0','1']))


NameError: ignored

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import NearestCentroid
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression




names = ["Logistic Regression", "KNN","Linear SVC","DC","SGD","RF","LinearSVC with L1-based feature selection", 
         "Bernoulli NB", "Ridge Classifier", "AdaBoost", "Perceptron","Passive-Aggresive", "Nearest C4entroid"]
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(),
    LinearSVC(),
    DecisionTreeClassifier(),
    SGDClassifier(),
    RandomForestClassifier(),
    Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))),
  ('classification', LinearSVC(penalty="l2"))]),
    BernoulliNB(),
    RidgeClassifier(),
    AdaBoostClassifier(),
    Perceptron(),
    PassiveAggressiveClassifier(),
    NearestCentroid()
    ]
zipped_clf = zip(names,classifiers)



```
`# This is formatted as code`
```

names = ["Logistic Regression", "KNN"]
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(),
    ]
zipped_clf = zip(names,classifiers)

In [ ]:
from time import time
from sklearn import metrics


def accuracy_summary(pipeline, x_train1, y_train1,x_test1,y_test1,tx_test1,ty_test1):#, x_valid1, y_valid1,
    t0 = time()
    sentiment_fit = pipeline.fit(x_train1, y_train1)
    y_pred = sentiment_fit.predict(tx_test1)
    train_test_time = time() - t0
    #accuracy = accuracy_score(y_test, y_pred)
    accuracy= (metrics.accuracy_score(ty_test1, y_pred))
    #print ("accuracy score: {0:.2f}%".format(accuracy*100))
    #print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    
  
    p= metrics.precision_score(ty_test1, y_pred,average= 'weighted')
    r = metrics.recall_score(ty_test1, y_pred,average= 'weighted')
    f=metrics.f1_score(ty_test1, y_pred, average= 'weighted')
    
    
    
    
    y_predt = sentiment_fit.predict(x_test1)
    #train_test_time = time() - t0
    #accuracy = accuracy_score(y_test, y_pred)
    accuracy2= (metrics.accuracy_score(y_test1, y_predt))
    #print ("accuracy score: {0:.2f}%".format(accuracy*100))
    #print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    
  
    tp= metrics.precision_score(y_test1, y_predt,average= 'weighted')
    tr = metrics.recall_score(y_test1, y_predt,average= 'weighted')
    tf=metrics.f1_score(y_test1, y_predt, average= 'weighted')

    
    
    
    ty_pred = sentiment_fit.predict(tx_test1)
    #train_test_time = time() - t0
    #accuracy = accuracy_score(y_test, y_pred)
    accuracy3= (metrics.accuracy_score(ty_test1, ty_pred))
    #print ("accuracy score: {0:.2f}%".format(accuracy*100))
    #print ("train and test time: {0:.2f}s".format(train_test_time))
    print ("-"*80)
    
  
    ttp= metrics.precision_score(ty_test1,ty_pred,average= 'weighted')
    ttr = metrics.recall_score(ty_test1, ty_pred,average= 'weighted')
    ttf=metrics.f1_score(ty_test1, ty_pred, average= 'weighted')

    
    
    
    
    return accuracy, p,r,f,accuracy2,tp,tr,tf,accuracy3,ttp,ttr,ttf


In [ ]:
def classifier_comparator():
    result1 = []
    result2 = []
    result3 = []

    classifier=zipped_clf
    
    for n,c in classifier:
        checker_pipeline = Pipeline([
                     ('classifier', c)
        ])
        print ("Validation result for {}".format(n))
        print (c)
        #clf_accuracy,p,r,f,tt_time = accuracy_summary(checker_pipeline, train_vecs_dbow, y_train, validation_vecs_dbow, y_valid)
        #clf_accuracy,p,r,f,tt_time = accuracy_summary(checker_pipeline, train_vecs_dmc, y_train, validation_vecs_dmc, y_valid)
        a1,p,r,f,a2,p2,r2,f2,a3,p3,r3,f3 = accuracy_summary(checker_pipeline,train_vecs_w2v,y_train,test_vecs_w2v,y_test,test_vecs_w2v,y_test)#y_valid

        result1.append((n,a1,p,r,f))
        result2.append((n,a2,p2,r2,f2))
        result3.append((n,a3,p3,r3,f3))

    return result1,result2,result3

trigram_result,trigram_result2,trigram_result3 = classifier_comparator()
#trigram_result = classifier_comparator()
#trigram_result = classifier_comparator()

from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.DataFrame(trigram_result)
df.columns=['classifier','acc','p','r','f1']

 
writer = ExcelWriter('FB_hi_w2vec_cbow.xlsx',engine='openpyxl')
df.to_excel(writer,'valid',index=False)
writer.save()

df = pd.DataFrame(trigram_result2)
df.columns=['classifier','acc','p','r','f1']

 
#writer = ExcelWriter('FB_English_valid_fasttext_cbow.xlsx',engine='openpyxl')
df.to_excel(writer,'test',index=False)
writer.save()

df = pd.DataFrame(trigram_result3)
df.columns=['classifier','acc','p','r','f1']

 
#writer = ExcelWriter('FB_English_valid_fasttext_cbow.xlsx',engine='openpyxl')
df.to_excel(writer,'ttransfer',index=False)
#writer.save()
df.head()

Validation result for Logistic Regression
LogisticRegression()


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for KNN
KNeighborsClassifier()
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for Linear SVC
LinearSVC()


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for DC
DecisionTreeClassifier()
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for SGD
SGDClassifier()
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for RF
RandomForestClassifier()
--------------------------------------------------------------------------------
------------------------------------------------

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for Bernoulli NB
BernoulliNB()
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for Ridge Classifier
RidgeClassifier()
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation result for AdaBoost
AdaBoostClassifier()
--------------------------------------------------------------------------------
--------------------------------

,classifier,acc,p,r,f1
0,Logistic Regression,0.707704,0.684602,0.707704,0.667816
1,KNN,0.651057,0.640367,0.651057,0.644808
2,Linear SVC,0.657855,0.637644,0.657855,0.644092
3,DC,0.595921,0.599710,0.595921,0.597745
4,SGD,0.678248,0.657683,0.678248,0.662765


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

#encoder = LabelEncoder()
#encoder.fit(y_valid)
#vencoded_Y = encoder.transform(y_valid)
# convert integers to dummy variables (i.e. one hot encoded)
#vdummy_y = np_utils.to_categorical(vencoded_Y)
#vdummy_y



encoder = LabelEncoder()
encoder.fit(y_test)
tencoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
tdummy_y = np_utils.to_categorical(tencoded_Y)
tdummy_y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
from keras.models import Sequential
model = Sequential()
model.add(Dense(7, activation='relu', input_dim=400))
#model.add(Dense(256, activation='relu', input_dim=300))
#model.add(Dense(256, activation='relu', input_dim=300))

model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v,dummy_y, epochs=12, batch_size=8, verbose=2)#validation_data=(valid_vecs_w2v, vdummy_y),

Epoch 1/12
1490/1490 - 2s - loss: 0.6622 - accuracy: 0.6475 - 2s/epoch - 2ms/step
Epoch 2/12
1490/1490 - 2s - loss: 0.5894 - accuracy: 0.6937 - 2s/epoch - 1ms/step
Epoch 3/12
1490/1490 - 2s - loss: 0.5819 - accuracy: 0.6982 - 2s/epoch - 1ms/step
Epoch 4/12
1490/1490 - 2s - loss: 0.5762 - accuracy: 0.6986 - 2s/epoch - 1ms/step
Epoch 5/12
1490/1490 - 2s - loss: 0.5718 - accuracy: 0.7066 - 2s/epoch - 1ms/step
Epoch 6/12
1490/1490 - 2s - loss: 0.5699 - accuracy: 0.7049 - 2s/epoch - 1ms/step
Epoch 7/12
1490/1490 - 2s - loss: 0.5659 - accuracy: 0.7086 - 2s/epoch - 1ms/step
Epoch 8/12
1490/1490 - 2s - loss: 0.5626 - accuracy: 0.7113 - 2s/epoch - 1ms/step
Epoch 9/12
1490/1490 - 2s - loss: 0.5593 - accuracy: 0.7124 - 2s/epoch - 1ms/step
Epoch 10/12
1490/1490 - 2s - loss: 0.5597 - accuracy: 0.7127 - 2s/epoch - 1ms/step
Epoch 11/12
1490/1490 - 2s - loss: 0.5589 - accuracy: 0.7137 - 2s/epoch - 1ms/step
Epoch 12/12
1490/1490 - 2s - loss: 0.5560 - accuracy: 0.7206 - 2s/epoch - 1ms/step


In [ ]:
#score = model.evaluate(valid_vecs_w2v, vdummy_y, batch_size=8, verbose=2)
#print (score[1])

score = model.evaluate(test_vecs_w2v, tdummy_y, batch_size=8, verbose=2)
print (score[1])



166/166 - 0s - loss: 0.5741 - accuracy: 0.7047 - 329ms/epoch - 2ms/step
0.7046827673912048


In [ ]:
from sklearn import metrics
import keras
#y_test=ty_test
y_pred_prob=model.predict(test_vecs_w2v)
y_pred_class = np.argmax(y_pred_prob,axis=1)
print(metrics.confusion_matrix(y_test, y_pred_class))
print (metrics.precision_score(y_test, y_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, y_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(y_test, y_pred_class,target_names=['0','1']))


[[102 321]
 [ 70 831]]
0.6803541853292817
0.7046827794561934
0.6604459383522667
              precision    recall  f1-score   support

           0       0.59      0.24      0.34       423
           1       0.72      0.92      0.81       901

    accuracy                           0.70      1324
   macro avg       0.66      0.58      0.58      1324
weighted avg       0.68      0.70      0.66      1324



In [ ]:
encoder = LabelEncoder()
encoder.fit(y_test)
tencoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
tdummy_y = np_utils.to_categorical(tencoded_Y)
tdummy_y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
from sklearn import metrics
import keras
score = model.evaluate(test_vecs_w2v, tdummy_y, batch_size=8, verbose=2)
print (score[1])

y_test=y_test
y_pred_prob=model.predict(test_vecs_w2v)
y_pred_class = np.argmax(y_pred_prob,axis=1)
print(metrics.confusion_matrix(y_test, y_pred_class))
print (metrics.precision_score(y_test, y_pred_class,average= 'weighted'))
print(metrics.recall_score(y_test, y_pred_class,average= 'weighted'))
print (metrics.f1_score(y_test, y_pred_class, average= 'weighted'))
#print (metrics.f1_score(y_test, y_pred_class))
print(metrics.classification_report(y_test, y_pred_class,target_names=['0','1']))


166/166 - 0s - loss: 0.5741 - accuracy: 0.7047 - 181ms/epoch - 1ms/step
0.7046827673912048
[[102 321]
 [ 70 831]]
0.6803541853292817
0.7046827794561934
0.6604459383522667
              precision    recall  f1-score   support

           0       0.59      0.24      0.34       423
           1       0.72      0.92      0.81       901

    accuracy                           0.70      1324
   macro avg       0.66      0.58      0.58      1324
weighted avg       0.68      0.70      0.66      1324

